In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from dataset import MyDataset

# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

dataset = MyDataset("dataset.csv")
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

def normalize(x):
        iterations_range = (5000, 120000)
        processes_range = (1, 8)
        length_of_message_range = (50, 500)
        min = torch.tensor([iterations_range[0], processes_range[0], length_of_message_range[0]]).to(device)
        max = torch.tensor([iterations_range[1], processes_range[1], length_of_message_range[1]]).to(device)
        x -= min
        x /= max
        return x

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleAutoencoder(nn.Module):
    def __init__(self):
        super(SimpleAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(3, 128),
            nn.ReLU(True),
            nn.Linear(128, 2)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(True),
            nn.Linear(128, 3),
            nn.Sigmoid()  # Use Sigmoid to ensure output is between 0 and 1 (since our data is normalized)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [3]:
# Define the model
model = SimpleAutoencoder().to(device)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500  # You can adjust this

for epoch in range(num_epochs):
    for input, _ in dataloader:
        # normalize the input
        input = normalize(input.to(device))
        # Forward pass
        output = model(input)
        loss = criterion(output, input)
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Log progress
    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [23/500], Loss: 0.0025
Epoch [123/500], Loss: 0.0010
Epoch [223/500], Loss: 0.0010
Epoch [323/500], Loss: 0.0007
Epoch [423/500], Loss: 0.0006


In [4]:
example_input, _ = next(iter(dataloader))
example_input = example_input.to(device)
example_input = normalize(example_input)

with torch.no_grad():
    encoded = model.encoder(example_input)
    decoded = model.decoder(encoded)

print(f'Input: {example_input[0]}')
print(f'Encoded: {encoded[0]}')
print(f'Decoded: {decoded[0]}')

Input: tensor([0.0833, 0.2500, 0.0000], device='cuda:0')
Encoded: tensor([-1.0955, -0.1517], device='cuda:0')
Decoded: tensor([0.1082, 0.2431, 0.0045], device='cuda:0')


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PredictionNN(nn.Module):
    def __init__(self):
        super(PredictionNN, self).__init__()
        # Input layer takes the 2D output from the encoder
        self.layer1 = nn.Linear(2, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 128)
        # Adjusted output layer to produce a vector of size 2
        self.output_layer = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        # No activation function here; can be added based on the specific requirements of the output
        x = F.relu(self.output_layer(x))
        return x


In [50]:
# Define the model
pred_model = PredictionNN().to(device)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(pred_model.parameters(), lr=0.0001)

# Training loop
num_epochs = 300 

for epoch in range(num_epochs):
    for input, label in dataloader:
        # normalize the input
        input = normalize(input.to(device))
        label = label.to(device)
        encoded = model.encoder(input)
        # Forward pass
        output = pred_model(encoded)
        loss = criterion(output, label)
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Log progress
    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/300], Loss: 428.7997
Epoch [101/300], Loss: 10.6818
Epoch [201/300], Loss: 1.4567


In [58]:
example_input, label = next(iter(dataloader))
example_input, label = example_input.to(device), label.to(device)
example_input = normalize(example_input)

with torch.no_grad():
    encoded = model.encoder(example_input)
    decoded = model.decoder(encoded)

    prediction = pred_model(encoded)

print(f'Input: {example_input[0]}')
print(f'Encoded: {encoded[0]}')
print(f'Decoded: {decoded[0]}')
print(f'Label: {label[0]}')
print(f'Prediction: {prediction[0]}')

Input: tensor([0.7083, 0.6250, 0.4000], device='cuda:0')
Encoded: tensor([ 1.0198, -0.4764], device='cuda:0')
Decoded: tensor([0.7099, 0.6244, 0.3986], device='cuda:0')
Label: tensor([37.9658,  2.0052], device='cuda:0')
Prediction: tensor([36.2300,  2.6684], device='cuda:0')
